In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objs as go

In [2]:
filename = "BSV.csv"
df = pd.read_csv(filename)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1258 entries, 0 to 1257
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1258 non-null   object 
 1   Open       1258 non-null   float64
 2   High       1258 non-null   float64
 3   Low        1258 non-null   float64
 4   Close      1258 non-null   float64
 5   Adj Close  1258 non-null   float64
 6   Volume     1258 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 68.9+ KB
None


In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_axis(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

In [4]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

1006
252


In [5]:
look_back = 10

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)


In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(10,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit(train_generator, epochs=num_epochs, verbose=1)

Epoch 1/50
50/50 [==============================] - 1s 3ms/step - loss: 131.2537
Epoch 2/50
50/50 [==============================] - 0s 4ms/step - loss: 0.5988
Epoch 3/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0600
Epoch 4/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0579
Epoch 5/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0627
Epoch 6/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0559
Epoch 7/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0608
Epoch 8/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0597
Epoch 9/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0585
Epoch 10/50
50/50 [==============================] - 0s 3ms/step - loss: 0.0583
Epoch 11/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0581
Epoch 12/50
50/50 [==============================] - 0s 4ms/step - loss: 0.0589
Epoch 13/50
50/50 [============================

In [14]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

In [15]:
close_data = close_data.reshape((-1))
look_back = 10
def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


In [16]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace2, trace3], layout=layout)
fig.show()

## ADJUSTED THIS IS FOR PORTFOLIO OPTIMIZATION

In [17]:
adj_close_data = df["Adj Close"].values
adj_close_data = adj_close_data.reshape((-1))
look_back = 10
def predict(num_prediction, model):
    prediction_list = adj_close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 30
adj_forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)



#--------------------------

adj_close_train = adj_close_data[:split]
adj_close_test = adj_close_data[split:]

adj_close_train = adj_close_train.reshape((-1))
adj_close_test = adj_close_test.reshape((-1))
adj_prediction = adj_forecast.reshape((-1))

trace1 = go.Scatter(
    x = forecast_dates,
    y = adj_close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = adj_prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = adj_close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[ trace2, trace3], layout=layout)
fig.show()

## ADDING ALL INFO INTO A FILE

In [18]:
#get all close prices values with predicted values (for protfolio prediction)
import datetime

indexes = pd.date_range(start='31/12/2021', end='1/30/2022')

newdf = pd.DataFrame(indexes, columns = ["Date"])
newdf["Close"] = forecast
newdf["Adj Close"] = adj_forecast
dffinal = df.loc[:,["Date" ,"Close","Adj Close"]]
dffinal = dffinal.append(newdf)
dffinal['Date'] = pd.to_datetime(dffinal['Date'])
dffinal

,Date,Close,Adj Close
2017-01-03 00:00:00,2017-01-03,79.459999,72.374512
2017-01-04 00:00:00,2017-01-04,79.459999,72.374512
2017-01-05 00:00:00,2017-01-05,79.580002,72.483833
2017-01-06 00:00:00,2017-01-06,79.489998,72.401855
2017-01-09 00:00:00,2017-01-09,79.580002,72.483833
...,...,...,...
26,2022-01-26,80.864494,80.606308
27,2022-01-27,80.864799,80.603691
28,2022-01-28,80.863533,80.600838
29,2022-01-29,80.860817,80.597763


In [19]:
dffinal.to_csv("BSVpredict.csv", index = False)

## PORTFOLIO OPTIMIZATION STATS

In [28]:
dffinal = pd.read_csv("BSVpredict.csv")

In [29]:
dffinal.set_axis(dffinal['Date'], inplace=True)
dffinal.drop(columns=["Date"],inplace = True)

In [30]:
#this are the predicted valyes 

dffinal.tail(60)

,Close,Adj Close
Date,,
2021-11-18,81.309998,80.801636
2021-11-19,81.269997,80.761894
2021-11-22,81.099998,80.592957
2021-11-23,81.080002,80.573082
2021-11-24,81.010002,80.503532
2021-11-26,81.300003,80.791718
2021-11-29,81.320000,80.811584
2021-11-30,81.320000,80.811584
2021-12-01,81.260002,80.823524
